In [ ]:
import numpy as np
import pandas as pd
import datetime
import torch
import torch.utils
from datetime import timezone

In [ ]:
# import 
from Trade_data_loader import *

In [ ]:
train_set = TradeDataset('train', data_dir='/tgbn-trade/tgbn-trade_final.csv')
test_set = TradeDataset('test',  data_dir='/tgbn-trade/tgbn-trade_final.csv')
initial_embeddings = np.random.randn (train_set.N_nodes, 32)
A_initial = train_set.get_Adjacency()

In [ ]:
# Verify sorting
def verify_sorted(events):
    timestamps = [event[3] for event in events]
    return all(timestamps[i] <= timestamps[i + 1] for i in range(len(timestamps) - 1))
assert verify_sorted(train_set.all_events), "Training set events are not sorted by timestamp."
assert verify_sorted(test_set.all_events), "Test set events are not sorted by timestamp."

In [ ]:
print("Train set preview (first 5 events):")
for event in train_set.all_events[:5]:
    print(event)

print("\nTest set preview (first 5 events):")
for event in test_set.all_events[:5]:
    print(event)

In [ ]:
from DyRep import *

In [ ]:
from utils import *

In [ ]:
import torch
from torch.utils.data import DataLoader

N_nodes = A_initial.shape[0]
if A_initial.ndim == 1 and A_initial.size == N_nodes * N_nodes:
    A_initial = A_initial.reshape(N_nodes, N_nodes)[:, :, None]  # Reshape and add relationship type dimension
elif A_initial.ndim == 1:  # If it's just a vector that doesn't match the expected size
    # Initialize A_initial as a zero matrix with an extra dimension for types
    A_initial = np.zeros((N_nodes, N_nodes, 1))
n_assoc_types,n_event_types = 1, 1
n_relations = n_assoc_types + n_event_types

Adj_all = train_set.get_Adjacency()[0]

if not isinstance(Adj_all, list):
    Adj_all = [Adj_all]

node_degree_global = []
for rel, A in enumerate(Adj_all):
    node_degree_global.append(np.zeros(A.shape[0]))
    for u in range(A.shape[0]):
        node_degree_global[rel][u] = np.sum(A[u])

Adj_all = Adj_all[0]
print("A_initial dimensions:", A_initial.shape)

In [ ]:
# # 2-hops model    
# model = DyRep_update(
#     node_embeddings=initial_embeddings,
#     A_initial=A_initial,
#     N_surv_samples=5,
#     n_hidden=32,
#     node_degree_global= node_degree_global,
#     N_hops=2,
#     with_attributes=True,
#     gamma=0.9,
# )    

In [ ]:
# 2-hops model    
model = DyRep_update(
    node_embeddings=initial_embeddings,
    A_initial=A_initial,
    N_surv_samples=5,
    n_hidden=32,
    node_degree_global= node_degree_global,
    N_hops=2,
)    

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=4096, shuffle=False)
test_loader = DataLoader(test_set, batch_size=4096, shuffle=False)

In [ ]:
import time

In [ ]:
def test(model, n_test_batches=10, epoch=0):
    model.eval()
    loss = 0
    losses =[ [np.Inf, 0], [np.Inf, 0] ]
    n_samples = 0
    # Time slots with 10 days intervals as in the DyRep paper
    timeslots = [t.toordinal() for t in test_loader.dataset.TEST_TIMESLOTS]
    event_types = list(test_loader.dataset.event_types_num.keys()) #['comm', 'assoc']
    # sort it by k
    for event_t in test_loader.dataset.event_types_num:
        event_types[test_loader.dataset.event_types_num[event_t]] = event_t

    event_types += ['Com']

    mar, hits_10 = {}, {}
    for event_t in event_types:
        mar[event_t] = []
        hits_10[event_t] = []
        for c, slot in enumerate(timeslots):
            mar[event_t].append([])
            hits_10[event_t].append([])


    start = time.time()
    with torch.no_grad():
        
        import datetime
        #from datetime import datetime, timezone 
        for batch_idx, data in enumerate(test_loader):
            data[2] = data[2].float()
            data[4] = data[4].double()
            data[5] = data[5].double()
            output = model(data)
            loss += (-torch.sum(torch.log(output[0]) + 1e-10) + torch.sum(output[1])).item()
            for i in range(len(losses)):
                m1 = output[i].min()
                m2 = output[i].max()
                if m1 < losses[i][0]:
                    losses[i][0] = m1
                if m2 > losses[i][1]:
                    losses[i][1] = m2
            n_samples += 1
            A_pred, Survival_term = output[2]
            u, v, k = data[0], data[1], data[3]

            time_cur = data[5]
            m, h = MAR(A_pred, u, v, k, Survival_term=Survival_term)
            assert len(time_cur) == len(m) == len(h) == len(k)
            for t, m, h, k_ in zip(time_cur, m, h, k):
                d = datetime.datetime.fromtimestamp(t.item()).toordinal()
                event_t = event_types[k_.item()]
                for c, slot in enumerate(timeslots):
                    if d <= slot:
                        mar[event_t][c].append(m)
                        hits_10[event_t][c].append(h)
                        if k_ > 0:
                            mar['Com'][c].append(m)
                            hits_10['Com'][c].append(h)
                        if c > 0:
                            assert slot > timeslots[c-1] and d > timeslots[c-1], (d, slot, timeslots[c-1])
                        break

            if batch_idx % 10 == 0:
                print('test', batch_idx)

            if n_test_batches is not None and batch_idx >= n_test_batches - 1:
                break

    time_iter = time.time() - start

    print('\nTEST batch={}/{}, loss={:.3f}, psi={}, loss1 min/max={:.4f}/{:.4f}, '
          'loss2 min/max={:.4f}/{:.4f}, integral time stamps={}, sec/iter={:.4f}'.
          format(batch_idx + 1, len(test_loader), (loss / n_samples),
                 [model.psi[c].item() for c in range(len(model.psi))],
                 losses[0][0], losses[0][1], losses[1][0], losses[1][1],
                 len(model.Lambda_dict), time_iter / (batch_idx + 1)))
    print(mar)
    # Report results for different time slots in the test set
    for c, slot in enumerate(timeslots):
        s = 'Slot {}: '.format(c)
        for event_t in event_types:
            sfx = '' if event_t == event_types[-1] else ', '
            if len(mar[event_t][c]) > 0:
                s += '{} ({} events): MAR={:.2f}+-{:.2f}, HITS_10={:.3f}+-{:.3f}'.\
                    format(event_t, len(mar[event_t][c]), np.mean(mar[event_t][c]), np.std(mar[event_t][c]),
                            np.mean(hits_10[event_t][c]), np.std(hits_10[event_t][c]))
            else:
                s += '{} (no events)'.format(event_t)
            s += sfx
        print(s)

    mar_all, hits_10_all = {}, {}
    for event_t in event_types:
        mar_all[event_t] = []
        hits_10_all[event_t] = []
        for c, slot in enumerate(timeslots):
            mar_all[event_t].extend(mar[event_t][c])
            hits_10_all[event_t].extend(hits_10[event_t][c])

    s = 'Epoch {}: results per event type for all test time slots: \n'.format(epoch)
    print(''.join(['-']*100))
    for event_t in event_types:
        if len(mar_all[event_t]) > 0:
            s += '====== {:10s}\t ({:7s} events): \tMAR={:.2f}+-{:.2f}\t HITS_10={:.3f}+-{:.3f}'.format(
                str(event_t),  # Ensure event_t is a string
                str(len(mar_all[event_t])),  # Ensure this is also a string if it isn't already
                np.mean(mar_all[event_t]),
                np.std(mar_all[event_t]),
                np.mean(hits_10_all[event_t]),
                np.std(hits_10_all[event_t])
            )
        else:
            s += '====== {:10s}\t (no events)'.format(str(event_t))  # Ensure event_t is a string
        if event_t != event_types[-1]:
            s += '\n'
    print(s)
    print(''.join(['-'] * 100))

    return mar_all, hits_10_all, loss / n_samples

In [ ]:
params_main = []
for name, param in model.named_parameters():
    if param.requires_grad:
        params_main.append(param)

In [ ]:
optimizer = torch.optim.Adam([{"params": params_main, "weight_decay":0}], lr=0.0002, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, '10', gamma=0.5)

In [ ]:
time_bar = np.zeros((N_nodes, 1)) + train_set.FIRST_DATE.timestamp()

In [ ]:
import copy
def get_temporal_variables():
    variables = {}
    variables['time_bar'] = copy.deepcopy(time_bar)
    variables['node_degree_global'] = copy.deepcopy(node_degree_global)
    variables['time_keys'] = copy.deepcopy(model.time_keys)
    variables['z'] = model.z.clone()
    variables['S'] = model.S.clone()
    variables['A'] = model.A.clone()
    variables['Lambda_dict'] = model.Lambda_dict.clone()
    return variables

In [ ]:
def set_temporal_variables(variables, model, train_loader, test_loader):
    time_bar = copy.deepcopy(variables['time_bar'])
    train_loader.dataset.time_bar = time_bar
    test_loader.dataset.time_bar = time_bar
    model.node_degree_global = copy.deepcopy(variables['node_degree_global'])
    model.time_keys = copy.deepcopy(variables['time_keys'])
    model.z = variables['z'].clone()
    model.S = variables['S'].clone()
    model.A = variables['A'].clone()
    model.Lambda_dict = variables['Lambda_dict'].clone()
    return time_bar

In [ ]:
epoch_start = 1
# 记得改回5
epochs = 1
batch_start = 0
batch_size = 4096
weight = 1
log_interval = 20
loss_l,losses_events, losses_nonevents, losses_sum = [], [], [], []
test_MAR, test_HITS10, test_loss = [], [], []

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=True)

In [ ]:
for epoch in range(epoch_start, epochs + 1):
    time_bar = np.zeros((train_loader.dataset.N_nodes, 1)) + train_loader.dataset.FIRST_DATE.timestamp()
    train_loader.dataset.time_bar = time_bar
    test_loader.dataset.time_bar = time_bar

    if epoch > epoch_start:
        _, node_degree_global = initialize_state(
            dataset=train_loader.dataset,
            model=model,
            node_embeddings=initial_embeddings,
            keepS=epoch > 1
        )
        model.node_degree_global = node_degree_global

    start = time.time()

    for batch_idx, data_batch in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()

        # Ensure the data is in the correct format
        data_batch[2] = data_batch[2].float()
        data_batch[4] = data_batch[4].double()
        data_batch[5] = data_batch[5].double()

        output = model(data_batch)
        losses = [-torch.sum(torch.log(output[0]) + 1e-10), weight * torch.sum(output[1])]

        if len(output) > 3 and output[-1] is not None:
            losses.extend(output[-1])

        loss = torch.sum(torch.stack(losses)) / batch_size
        loss.backward()
        nn.utils.clip_grad_value_(model.parameters(), 100)
        optimizer.step()
        losses_events.append(losses[0].item())
        losses_nonevents.append(losses[1].item())
        losses_sum.append(loss.item())

        model.psi.data = torch.clamp(model.psi.data, 1e-1, 1e+3)

        time_iter = time.time() - start
        model.z = model.z.detach()
        model.S = model.S.detach()

        if (batch_idx + 1) % log_interval == 0 or batch_idx == len(train_loader) - 1:
            print(f'\nTRAIN epoch={epoch}/{epochs}, batch={batch_idx + 1}/{len(train_loader)}, '
                  f'sec/iter: {time_iter / (batch_idx + 1):.4f}, loss={loss.item():.3f}, '
                  f'loss components: {[l.item() for l in losses]}')
            loss_l.append(loss.item())

            variables = get_temporal_variables()
            print('time', datetime.datetime.fromtimestamp(np.max(time_bar)))

            result = test(model, n_test_batches=None if batch_idx == len(train_loader) - 1 else 10, epoch=epoch)
            test_MAR.append(np.mean(result[0].get('Com', [np.nan])))
            test_HITS10.append(np.mean(result[1].get('Com', [np.nan])))
            test_loss.append(result[2])
            time_bar = set_temporal_variables(variables, model, train_loader, test_loader)

    scheduler.step()

model.load_state_dict(torch.load('checkpoint.pt'))
print('Training complete with best model loaded')
